#### 가성비 스코어 계산 + TOP20 추천

In [1]:
import pandas as pd
from xgboost import XGBRegressor

df = pd.read_csv("./올영_세일.csv", encoding="utf-8")

def to_num(x):
    if pd.isna(x):
        return None
    x = str(x).replace("원","").replace(",","").replace("~","").strip()
    return int(x) if x.isdigit() else None

df["price_num"] = df["price"].apply(to_num)
df["orig_price_num"] = df["orig_price"].apply(to_num)
df["discount_rate"] = (df["orig_price_num"] - df["price_num"]) / df["orig_price_num"]
df["discount_rate"] = df["discount_rate"].fillna(0)

df["rank"] = df.index + 1

df_model = df[["price_num", "orig_price_num", "discount_rate", "rank"]].dropna()

X = df_model[["price_num", "orig_price_num", "discount_rate"]]
y = df_model["rank"]


model = XGBRegressor(
    n_estimators=400,
    learning_rate=0.05,
    max_depth=4,
    subsample=0.8,
    colsample_bytree=0.8,
    random_state=42,
    objective="reg:squarederror"
)

model.fit(X, y)


df["pred_rank"] = model.predict(X)


df["popularity_score"] = 1 / df["pred_rank"]

df["value_score"] = df["popularity_score"] / df["price_num"]

top20_value = df.sort_values("value_score", ascending=False).head(20)

top20_value[["name", "price_num", "discount_rate", "pred_rank", "value_score", "url"]]

,name,price_num,discount_rate,pred_rank,value_score,url
51,메디힐\n[14년연속 누적판매 1위] 메디힐 에센셜 마스크팩 1매 고기능 7종 택1,950,0.525000,230.766418,4.561459e-06,https://www.oliveyoung.co.kr/store/goods/getGo...
25,딜라이트 프로젝트\n[어워즈특가] 딜라이트 프로젝트 베이글칩 어워즈 한정기획 7종 ...,2500,0.074074,186.337158,2.146646e-06,https://www.oliveyoung.co.kr/store/goods/getGo...
12,딜라이트 프로젝트\n딜라이트 프로젝트 베이글칩 모음전,2500,0.074074,186.337158,2.146646e-06,https://www.oliveyoung.co.kr/store/goods/getGo...
380,딜라이트 프로젝트\n딜라이트 프로젝트 베이글칩 8종 택1,2500,0.074074,186.337158,2.146646e-06,https://www.oliveyoung.co.kr/store/goods/getGo...
209,쏘피\n[반값특가]쏘피 바디피트 내몸에 순한면 10종 택 1 (단품/번들),1330,0.300000,354.049011,2.123660e-06,https://www.oliveyoung.co.kr/store/goods/getGo...
658,좋은느낌\n좋은느낌 오리지널 울트라날개 생리대 9종 택 1 (소형 / 중형 / 대형...,1170,0.350000,629.292969,1.358192e-06,https://www.oliveyoung.co.kr/store/goods/getGo...
313,딜라이트 프로젝트\n[연말결산특가] 딜라이트 프로젝트 프레첼 3종 택1,2160,0.100000,368.671814,1.255759e-06,https://www.oliveyoung.co.kr/store/goods/getGo...
587,딜라이트 프로젝트\n딜라이트 프로젝트 프레첼 3종 택1,2160,0.100000,368.671814,1.255759e-06,https://www.oliveyoung.co.kr/store/goods/getGo...
826,듀이트리\n[밀착진정] 듀이트리 AC 딥 진정 마스크 1매 (열감/흔적),1190,0.603333,692.137573,1.214117e-06,https://www.oliveyoung.co.kr/store/goods/getGo...
648,케어플러스\n[99특가/1+1] 케어플러스 아랫배 찜질 패드 1매입,990,0.175000,865.400085,1.167207e-06,https://www.oliveyoung.co.kr/store/goods/getGo...


#### 할인율 변화 시 순위 시뮬레이터

In [2]:
def simulate_discount_effect(product_index, model, df):
    row = df.iloc[product_index].copy()
    
    print("▶ 상품명:", row["name"])
    print("▶ 현재 가격:", row["price_num"], "원")
    print("▶ 현재 할인율:", round(row["discount_rate"] * 100, 1), "%")
    print("-"*50)

    results = []

    for rate in [0.1, 0.2, 0.3, 0.4, 0.5]:
        new_price = int(row["orig_price_num"] * (1 - rate))
        features = [[new_price, row["orig_price_num"], rate]]

        pred_rank = model.predict(features)[0]

        results.append({
            "할인율": f"{int(rate*100)}%",
            "예상가격": new_price,
            "예상순위": round(pred_rank, 1)
        })

    return pd.DataFrame(results)

In [ ]:
📌 이걸로 가능한 응용

✔ 어떤 할인율에서 순위 개선 효율이 가장 큰지 분석

→ 세일 전략 추천 가능

✔ 브랜드별 최적 할인율 계산 가능

→ “이 브랜드는 30% 이상 해야 사람들 관심 받는다” 같은 분석

✔ 상품 추천 시스템에서 “할인 시 인기 상승률” 적용 가능